Version1 : best LB score

Version 2:  Increase to 400 rounds

Version 4 : 0.5 wowfattie + 0.25 BiGRU + 0.25 Elmo

Version 5 : 0.4 wow fattie + 0.3 EMA + 0.15 BiGRU + 0.15 Elmo 

Version 6: change to 700 rounds

Version 7 : Change to 500 rounds * 10 folds --> not much improve in the local CV score 

Version 9 Change to 700 rounds * 5 fold  AND  0.5 wow fattie + 0.4 EMA + 0.10 BiGRU + 0.10 Elmo ==> save oof and pred file as well 

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import os 

In [2]:
print(os.listdir("../input/ema-19032019"))


['pred_EMA.np.npy', 'custom.css', '__results__.html', 'submission.csv', '__output__.json', '__notebook__.ipynb', 'oof_EMA.np.npy']


In [3]:
# # train_EMA = np.load('../input/ema-19032019/oof_EMA.np.npy')
# train_EMA  =np.load('../input/ensembling-ndsc/ensembling_v2/oof_Elmo.np.npy') 
# train = pd.read_csv('../input/ndsc-beginner/train.csv')
# y = train['Category']
# from sklearn.metrics import accuracy_score
# oof_ = [np.argmax(preds) for preds in train_EMA]
# print("CV score: {:<8.5f}".format(accuracy_score(y, oof_)))

In [4]:
train_wowfattie = np.load('../input/ensembling-ndsc/ensembling_v2/oof_prob1.npy') #0.79 acc - 0.758 LB
train_EMA = np.load('../input/ema-19032019/oof_EMA.np.npy') #0.77 acc 
train_BiGRU = np.load('../input/ensembling-ndsc/ensembling_v2/oof_Bi-GRU-LSTM-CNN-Poolings-Fasttext.np.npy') #0.73 acc
train_elmo = np.load('../input/ensembling-ndsc/ensembling_v2/oof_Elmo.np.npy') #0.71 acc 

test_wowfattie = np.load('../input/ensembling-ndsc/ensembling_v2/pred_wowfattie.npy')
test_EMA  =np.load('../input/ema-19032019/pred_EMA.np.npy')
test_BiGRU = np.load('../input/ensembling-ndsc/ensembling_v2/pred_BiGRU.npy')
test_elmo = np.load('../input/ensembling-ndsc/ensembling_v2/pred_elmo.npy')

#------------------------------------
train_arr = np.zeros((666615, 58))
test_arr = np.zeros((172402, 58))

train_arr += 0.5*np.squeeze(train_wowfattie)
train_arr += 0.3*np.squeeze(train_EMA)
train_arr += 0.10*np.squeeze(train_BiGRU)
train_arr += 0.10*np.squeeze(train_elmo)

test_arr += 0.5*np.squeeze(test_wowfattie)
test_arr += 0.3*np.squeeze(test_EMA)
test_arr += 0.10*np.squeeze(test_BiGRU)
test_arr += 0.10*np.squeeze(test_elmo)

train_arr = pd.DataFrame(train_arr ,columns=['class_'+str(i) for i in range(58)])
test_arr = pd.DataFrame(test_arr ,columns=['class_'+str(i) for i in range(58)])

In [5]:
test_BiGRU.shape


(172402, 58)

In [6]:
train_wowfattie.shape

(666615, 58)

In [7]:
train = pd.read_csv('../input/ndsc-beginner/train.csv')
print(train.shape)
test = pd.read_csv('../input/ndsc-beginner/test.csv')
y = train['Category']
train = train[['itemid']]
test = test[['itemid']]

(666615, 4)


In [8]:
train = pd.concat([train,train_arr],axis= 1, sort = False)
test = pd.concat([test,test_arr], axis =1 ,sort = False )

del train_elmo,train_wowfattie
del test_elmo,test_wowfattie

In [9]:
# del sub_preds

In [10]:
#get number of feature and number of training rows
X_train = train
X_test = test

del train, test 

num_train, num_feature = X_train.shape

#Get parameters 
params = {
    'boosting_type': 'gbdt',
    'objective': 'softmax',
    'metric': 'multi_error',
    'num_class':58,
    'max_depth':20,
    'num_leaves': 40,
    'learning_rate': 0.02,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 100,
}


#------------------------------------------------------------
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

oof_preds = np.zeros((666615,58))
sub_preds = np.zeros((172402, 58))
feature_importance_df = pd.DataFrame()
feature_name = [col for col in X_train.columns]
print(feature_name)
print('Starting training...')
print(X_train.shape)
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train,y)):
    print('fold {} '.format(n_fold))
    trn_x, trn_y = X_train[feature_name].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X_train[feature_name].iloc[val_idx], y.iloc[val_idx]
    lgb_train = lgb.Dataset(trn_x, trn_y)
    lgb_eval = lgb.Dataset(val_x, val_y)

    gbm = lgb.train(params,
                lgb_train,
#                 num_boost_round = 2,
                num_boost_round=700,
                valid_sets=lgb_eval,
                early_stopping_rounds=100,
                   verbose_eval=10)
    
    oof_preds[val_idx] += gbm.predict(val_x, num_iteration=gbm.best_iteration) #get oof prediction
    print(oof_preds.shape)
    #predict on test set, take average
    sub_preds += gbm.predict(X_test[feature_name], num_iteration=gbm.best_iteration) / folds.n_splits 
    print (sub_preds.shape)
    #save the feature important 
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feature_name
    fold_importance_df["importance"] = np.log1p(gbm.feature_importance(
        importance_type='gain',
        iteration=gbm.best_iteration))
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    


['itemid', 'class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_40', 'class_41', 'class_42', 'class_43', 'class_44', 'class_45', 'class_46', 'class_47', 'class_48', 'class_49', 'class_50', 'class_51', 'class_52', 'class_53', 'class_54', 'class_55', 'class_56', 'class_57']
Starting training...
(666615, 59)
fold 0 
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's multi_error: 0.325327
[20]	valid_0's multi_error: 0.21445
[30]	valid_0's multi_error: 0.204071
[40]	valid_0's multi_error: 0.201213
[50]	valid_0's multi_error: 0.200208
[60]	valid_0's

In [11]:
X_test.head()

,itemid,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13,class_14,class_15,class_16,class_17,class_18,class_19,class_20,class_21,class_22,class_23,class_24,class_25,class_26,class_27,class_28,class_29,class_30,class_31,class_32,class_33,class_34,class_35,class_36,class_37,class_38,class_39,class_40,class_41,class_42,class_43,class_44,class_45,class_46,class_47,class_48,class_49,class_50,class_51,class_52,class_53,class_54,class_55,class_56,class_57
0,370855998,0.001850,0.044917,0.000970,0.006066,0.056000,0.835339,0.000226,0.006140,0.001603,0.009879,0.000071,0.000112,0.001209,0.000061,0.000035,4.808243e-06,0.000071,4.933851e-08,7.974089e-07,2.976547e-07,2.544489e-07,6.393340e-08,7.584292e-08,4.236838e-08,8.034726e-08,3.989470e-07,6.489148e-07,4.044882e-07,1.662497e-07,2.115014e-07,4.642222e-08,1.020600e-06,5.997463e-07,4.650641e-07,4.364452e-07,1.664378e-06,4.391708e-06,4.041247e-07,1.446184e-07,1.349943e-07,4.748361e-09,4.085313e-07,7.030420e-08,1.926955e-06,5.445354e-08,2.145683e-07,1.676140e-08,5.811766e-07,1.089867e-08,2.152762e-08,1.762810e-08,8.626821e-08,4.335477e-08,2.702054e-07,2.366685e-08,1.047635e-09,2.995880e-09,7.390539e-10
1,637234604,0.000178,0.031456,0.000511,0.006598,0.041839,0.890569,0.000024,0.002722,0.000702,0.002523,0.000006,0.000021,0.000268,0.000006,0.000003,3.911553e-07,0.000010,4.061625e-09,1.661847e-07,6.311369e-08,1.813656e-08,9.835560e-09,6.249059e-09,9.773802e-10,2.580126e-08,7.004701e-08,5.392203e-08,1.108097e-07,4.787787e-08,4.479295e-08,1.153467e-08,7.195673e-08,6.990842e-08,4.861819e-08,5.662393e-08,1.111461e-07,1.341164e-06,9.956766e-09,1.556156e-08,3.567944e-09,2.394568e-10,3.870611e-08,1.306767e-08,8.790076e-08,4.194320e-09,8.526442e-08,1.313812e-09,6.216684e-08,1.255805e-09,1.028245e-08,2.059014e-09,1.740265e-08,2.137355e-09,2.125902e-09,2.910214e-09,8.677417e-10,7.608550e-11,4.802772e-10
2,690282890,0.002819,0.033465,0.000358,0.004474,0.110529,0.835323,0.000025,0.001751,0.000370,0.007101,0.000043,0.000494,0.000206,0.000094,0.000010,1.118492e-06,0.000160,1.365760e-07,6.218940e-07,2.145429e-07,1.254660e-07,2.834860e-08,7.107559e-07,1.681893e-08,8.517255e-09,3.212414e-07,3.505192e-07,2.231795e-07,3.561056e-07,1.445951e-07,4.547634e-08,3.331032e-07,1.351771e-07,4.544003e-08,1.908759e-07,1.321776e-05,3.192979e-06,7.172879e-08,1.098006e-07,4.413486e-08,1.043498e-09,5.853772e-08,4.435656e-08,9.612162e-08,7.217265e-09,1.349682e-07,1.424652e-08,2.379910e-08,5.371527e-09,2.228502e-07,3.573852e-08,4.176700e-08,4.948529e-09,1.030882e-07,5.053217e-09,6.739011e-10,3.083631e-10,1.201587e-09
3,930913462,0.001192,0.057847,0.000337,0.037252,0.384195,0.328754,0.000093,0.004284,0.003023,0.125021,0.000074,0.000176,0.008168,0.000134,0.000043,1.315830e-05,0.001303,4.019498e-07,1.021202e-05,9.511627e-06,4.421665e-07,3.866331e-07,3.333920e-07,2.036495e-07,3.498404e-07,5.910052e-07,1.336281e-06,1.501932e-06,4.309844e-07,1.717673e-07,1.188047e-07,5.449635e-07,3.849872e-07,2.281406e-07,1.675680e-07,3.856786e-07,4.884625e-07,1.871354e-07,2.853231e-08,1.256802e-07,2.350138e-09,1.007293e-06,2.128278e-07,3.773620e-07,3.336822e-07,2.420012e-07,2.489655e-08,7.962221e-07,2.205565e-08,1.141978e-06,3.959483e-08,9.232079e-08,7.553995e-09,4.362959e-08,6.632851e-08,3.528037e-09,5.289797e-10,5.085453e-10
4,1039280071,0.000261,0.037601,0.000116,0.002205,0.025048,0.910403,0.000030,0.001283,0.000134,0.003669,0.000022,0.000017,0.000192,0.000020,0.000010,3.889596e-07,0.000012,7.305369e-09,5.720122e-07,4.924130e-08,3.707356e-08,1.709399e-08,1.667512e-08,1.004745e-08,3.023639e-08,1.967469e-07,1.259956e-07,6.517855e-08,4.940409e-08,5.605551e-08,1.328595e-08,4.589715e-08,1.125724e-07,1.354421e-08,6.350568e-08,1.331577e-07,3.322273e-07,1.527281e-08,3.108906e-08,3.877804e-09,9.121940e-11,2.154521e-07,1.426709e-08,9.953765e-08,2.471264e-09,4.012296e-08,3.387956e-09,2.798455e-08,1.542645e-09,1.217091e-08,1.011527e-09,2.353609e-07,1.314023e-09,3.571198e-08,2.052482e-09,3.993701

In [12]:
from sklearn.metrics import accuracy_score
oof_ = [np.argmax(preds) for preds in oof_preds]
cv_score = accuracy_score(y, oof_)
print("CV score: {:<8.5f}".format(accuracy_score(y, oof_)))

CV score: 0.81374 


In [13]:
sub_preds.shape

(172402, 58)

In [14]:
np.save('oof_stacking_'+str(cv_score) +'.np',oof_preds)
print('finished saving oof file')
np.save('pred_stacking_'+str(cv_score) +'.np',sub_preds)
print('finished saving pred file')


finished saving oof file
finished saving pred file


In [15]:
submission = pd.read_csv('../input/ndsc-beginner/data_info_val_sample_submission.csv')
y_te = [np.argmax(preds) for preds in sub_preds]
submission['Category'] = y_te
# submission['Category'] = sub_preds
submission.to_csv('submission_stack.csv',index = False)


In [16]:
submission.head()

,itemid,Category
0,370855998,5
1,637234604,5
2,690282890,5
3,930913462,4
4,1039280071,5


In [17]:
print(143243)

143243
